In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [ ]:
# Define the Language Model
class LanguageModel(nn.Module):
    def __init__(self, vocab_size, n_embd=384, n_head=8, n_layer=8, block_size=256, dropout=0.3):
        super(LanguageModel, self).__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.ModuleList([Block(n_embd, n_head, dropout) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)
        self.block_size = block_size

        self._init_weights()

    def _init_weights(self):
        for module in self.modules():
            if isinstance(module, nn.Linear):
                nn.init.normal_(module.weight, mean=0.0, std=0.02)
                if module.bias is not None:
                    nn.init.zeros_(module.bias)
            elif isinstance(module, nn.Embedding):
                nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, idx, targets=None):
        B, T = idx.shape
        tok_emb = self.token_embedding_table(idx)
        pos_emb = self.position_embedding_table(torch.arange(T, device=idx.device))
        x = tok_emb + pos_emb 
        for block in self.blocks:
            x = block(x)
        x = self.ln_f(x) 
        logits = self.lm_head(x)

        if targets is not None:
            loss = F.cross_entropy(logits.view(-1, logits.size(-1)), targets.view(-1))
            return logits, loss
        else:
            return logits

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -self.block_size:]
            logits = self(idx_cond)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)
        return idx


In [ ]:
# Define the Transformer Block
class Block(nn.Module):
    def __init__(self, n_embd, n_head, dropout=0.3):
        super(Block, self).__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size, dropout)
        self.ffwd = FeedForward(n_embd, dropout)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x


In [ ]:
# Define the Multi-Head Attention Mechanism
class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, head_size, dropout=0.3):
        super(MultiHeadAttention, self).__init__()
        self.heads = nn.ModuleList([Head(head_size, dropout) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size * num_heads, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out


In [ ]:
# Define a Single Attention Head
class Head(nn.Module):
    def __init__(self, head_size, dropout):
        super(Head, self).__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B, T, C = x.shape
        k = self.key(x) 
        q = self.query(x)
        wei = q @ k.transpose(-2, -1) * k.shape[-1] ** -0.5
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        wei = F.softmax(wei, dim=-1)
        wei = self.dropout(wei)
        v = self.value(x)
        out = wei @ v
        return out


In [ ]:
# Define the FeedForward Network
class FeedForward(nn.Module):
    def __init__(self, n_embd, dropout):
        super(FeedForward, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)


In [ ]:
# Function to Train the Model
def train(model, train_data, val_data, optimizer, max_iters, eval_interval, block_size, device):
    model.train()
    for iter in range(max_iters):
        if iter % eval_interval == 0 or iter == max_iters - 1:
            losses = evaluate(model, train_data, val_data, block_size, device)
            print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

        xb, yb = get_batch(train_data, block_size, device)
        logits, loss = model(xb, yb)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()


In [ ]:
# Function to Evaluate the Model
def evaluate(model, train_data, val_data, block_size, device):
    model.eval()
    with torch.no_grad():
        train_loss = calculate_loss(model, train_data, block_size, device)
        val_loss = calculate_loss(model, val_data, block_size, device)
    model.train()
    return {'train': train_loss, 'val': val_loss}


In [ ]:
# Function to Calculate Loss
def calculate_loss(model, data, block_size, device):
    losses = []
    for _ in range(eval_iters):
        xb, yb = get_batch(data, block_size, device)
        logits, loss = model(xb, yb)
        losses.append(loss.item())
    return sum(losses) / len(losses)


In [ ]:
# Function to Get a Batch of Data
def get_batch(data, block_size, device):
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i + block_size] for i in ix])
    y = torch.stack([data[i + 1:i + block_size + 1] for i in ix])
    return x.to(device), y.to(device)

In [2]:
# Function to Decode Token Indices to Text
def decode(indices):
    return ''.join([itos[i] for i in indices])

# Hyperparameters
batch_size = 64
block_size = 256
max_iters = 10000
eval_interval = 500
learning_rate = 1e-4
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 384
n_head = 8
n_layer = 8
dropout = 0.3

# Load Data
with open('/kaggle/input/anglo-american-poems/all_data.txt', 'r', encoding='utf-8') as f:
    text = f.read()

chars = sorted(list(set(text)))
vocab_size = len(chars)
stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for i, ch in enumerate(chars)}

# Train and Test Splits
data = torch.tensor([stoi[ch] for ch in text], dtype=torch.long)
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

# Instantiate Model and Optimizer
model = LanguageModel(vocab_size).to(device)
optimizer = optim.AdamW(model.parameters(), lr=learning_rate)

# Train the Model
train(model, train_data, val_data, optimizer, max_iters, eval_interval, block_size, device)

# Generate Text from the Trained Model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
generated_text = decode(model.generate(context, max_new_tokens=1000)[0].tolist())
print(generated_text)


step 0: train loss 4.6948, val loss 4.7104
step 500: train loss 2.2285, val loss 2.3328
step 1000: train loss 1.8897, val loss 1.9982
step 1500: train loss 1.7067, val loss 1.8170
step 2000: train loss 1.6030, val loss 1.7309
step 2500: train loss 1.5276, val loss 1.6659
step 3000: train loss 1.4767, val loss 1.6279
step 3500: train loss 1.4313, val loss 1.5997
step 4000: train loss 1.3874, val loss 1.5709
step 4500: train loss 1.3584, val loss 1.5510
step 5000: train loss 1.3335, val loss 1.5357
step 5500: train loss 1.3101, val loss 1.5276
step 6000: train loss 1.2900, val loss 1.5181
step 6500: train loss 1.2677, val loss 1.5123
step 7000: train loss 1.2472, val loss 1.5044
step 7500: train loss 1.2318, val loss 1.4988
step 8000: train loss 1.2147, val loss 1.4953
step 8500: train loss 1.1983, val loss 1.4929
step 9000: train loss 1.1819, val loss 1.4913
step 9500: train loss 1.1671, val loss 1.4906
step 9999: train loss 1.1543, val loss 1.4862

_Before I'd forget my three talked --